<a href="https://colab.research.google.com/github/nadinejackson1/climate-change-food-security-lstm/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries and  modules
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Set the folder path 
folder = "/content/drive/MyDrive/caltech-101/101_ObjectCategories"

In [3]:
# Define function to load images from folder

def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    labels = []
    for class_name in os.listdir(folder):
        path = os.path.join(folder, class_name)
        if os.path.isdir(path):
            for filename in os.listdir(path):
                img = cv2.imread(os.path.join(path, filename))
                if img is not None:
                    img = cv2.resize(img, target_size)
                    images.append(img)
                    labels.append(class_name)
    return images, labels

In [4]:
# Load and preprocess the dataset

images, labels = load_images_from_folder(folder)
images = np.array(images) / 255.0

if len(labels) == 0:
    print("No data found. Please check the dataset path.")
else:
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    num_classes = np.max(integer_encoded) + 1 if integer_encoded.size > 0 else 0
    labels = to_categorical(integer_encoded, num_classes=num_classes)

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [6]:
# Define the CNN model
def create_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]
model = create_cnn(input_shape, num_classes)


In [7]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
187/187 [==============================] - 19s 43ms/step - loss: 3.2836 - accuracy: 0.3186 - val_loss: 2.6421 - val_accuracy: 0.4216
Epoch 2/20
187/187 [==============================] - 4s 22ms/step - loss: 2.0651 - accuracy: 0.5316 - val_loss: 2.0653 - val_accuracy: 0.5422
Epoch 3/20
187/187 [==============================] - 4s 23ms/step - loss: 1.2316 - accuracy: 0.6933 - val_loss: 2.0295 - val_accuracy: 0.5556
Epoch 4/20
187/187 [==============================] - 5s 24ms/step - loss: 0.6235 - accuracy: 0.8270 - val_loss: 2.1145 - val_accuracy: 0.5851
Epoch 5/20
187/187 [==============================] - 7s 40ms/step - loss: 0.3254 - accuracy: 0.9120 - val_loss: 2.4975 - val_accuracy: 0.5912
Epoch 6/20
187/187 [==============================] - 35s 182ms/step - loss: 0.2195 - accuracy: 0.9370 - val_loss: 2.8972 - val_accuracy: 0.5945
Epoch 7/20
187/187 [==============================] - 4s 24ms/step - loss: 0.1376 - accuracy: 0.9625 - val_loss: 2.8607 - val_accuracy: 0.5

In [ ]:
# Evaluate the model on the test set:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")
